# Spotrac Scraping

In [6]:
import requests
import pandas as pd
from pathlib import Path

# We're just testing, but let's practice using our config-style variables
SEASON = "2024-25"
TARGET_URL = f"https://www.spotrac.com/nba/contracts"

# This is the path we eventually want to write to
# (We need to go up two parent directories from `notebooks/Tyler/`)
PROJECT_ROOT = Path().resolve().parent.parent
RAW_SALARY_FILE = PROJECT_ROOT / "data" / "raw" / "raw_player_salaries.csv"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Target URL: {TARGET_URL}")
print(f"Target File: {RAW_SALARY_FILE}")

Project Root: C:\Users\tyler\School\Learn Statistics\STA 160\Project
Target URL: https://www.spotrac.com/nba/contracts
Target File: C:\Users\tyler\School\Learn Statistics\STA 160\Project\data\raw\raw_player_salaries.csv


In [7]:
# Set a browser-like User-Agent so the site doesn't block us
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

try:
    response = requests.get(TARGET_URL, headers=headers)
    response.raise_for_status()  # This will raise an error if the request failed
    print("Success! Got the webpage.")
    html_content = response.text
except requests.exceptions.RequestException as e:
    print(f"Error fetching page: {e}")
    html_content = None

Success! Got the webpage.


In [8]:
# pd.read_html returns a LIST of all tables found on the page
# We need to inspect this list to find the one we want

if html_content:
    try:
        all_tables = pd.read_html(html_content)
        print(f"Found {len(all_tables)} tables on the page.")
        
        # This is a guess; we might need to look. 
        # Often the main table is the first one.
        salary_df_raw = all_tables[0] 
        
        print("--- Head of the first table found: ---")
        display(salary_df_raw.head())
        
    except ValueError as e:
        print(f"No tables found or error parsing: {e}")
        salary_df_raw = None
else:
    print("No HTML content to parse.")

Found 1 tables on the page.
--- Head of the first table found: ---


C:\Users\tyler\AppData\Local\Temp\ipykernel_21708\612414455.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_tables = pd.read_html(html_content)


,RK,Player,Pos,Team Currently With,Age At Signing,Start,End,Yrs,Value,AAV
0,1,Jayson Tatum,PF,BOS BOS,26,2025,2029,5,"$313,933,410","$62,786,682"
1,2,Jaylen Brown,SF,BOS BOS,26,2024,2028,5,"$285,393,640","$57,078,728"
2,3,Nikola Jokic,C,DEN DEN,27,2023,2027,5,"$276,122,630","$55,224,526"
3,4,Shai Gilgeous-Alexander,PG,OKC OKC,26,2027,2030,4,"$273,302,400","$68,325,600"
4,T5,Cade Cunningham,PG,DET DET,22,2025,2029,5,"$269,085,780","$53,817,156"


In [ ]:
if salary_df_raw is not None:
    try:
        
        final_df = salary_df_raw[['Player', 'AAV']].copy()
        
        # Rename them to match our contract
        final_df.rename(columns={
            'Player': 'Player_Name',
            'Base Salary': 'Salary'
        }, inplace=True)

        print("--- Cleaned and Finalized DataFrame: ---")
        display(final_df.head())

        print(f"size: {len(final_df)}")

    except KeyError as e:
        print(f"Error: A column name was wrong. You need to inspect the raw table.")
        print(f"Raw columns are: {salary_df_raw.columns}")
        final_df = None
else:
    print("Can't inspect, raw DataFrame is empty.")



--- Cleaned and Finalized DataFrame: ---


,Player_Name,AAV
0,Jayson Tatum,"$62,786,682"
1,Jaylen Brown,"$57,078,728"
2,Nikola Jokic,"$55,224,526"
3,Shai Gilgeous-Alexander,"$68,325,600"
4,Cade Cunningham,"$53,817,156"


size: 100


# Spotrac is not feasible. Let's try other sites.

# Wait a minute. There is different team endpoint.

In [15]:
import requests
import pandas as pd
from pathlib import Path
import io # To fix the FutureWarning
import time # To be a polite scraper

# This is the list of all 30 team slugs used by Spotrac
TEAM_SLUGS = [
    'atlanta-hawks', 'boston-celtics', 'brooklyn-nets', 'charlotte-hornets', 
    'chicago-bulls', 'cleveland-cavaliers', 'dallas-mavericks', 'denver-nuggets',
    'detroit-pistons', 'golden-state-warriors', 'houston-rockets', 'indiana-pacers',
    'los-angeles-clippers', 'los-angeles-lakers', 'memphis-grizzlies', 
    'miami-heat', 'milwaukee-bucks', 'minnesota-timberwolves', 'new-orleans-pelicans',
    'new-york-knicks', 'oklahoma-city-thunder', 'orlando-magic', 
    'philadelphia-76ers', 'phoenix-suns', 'portland-trail-blazers', 
    'sacramento-kings', 'san-antonio-spurs', 'toronto-raptors', 'utah-jazz', 
    'washington-wizards'
]

# Config vars for practice
SEASON = "2024-25"
SEASON_START_YEAR = SEASON.split('-')[0] # Will give us "2024"

PROJECT_ROOT = Path().resolve().parent.parent
RAW_SALARY_FILE = PROJECT_ROOT / "data" / "raw" / "raw_player_salaries.csv"

# Set browser headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

In [ ]:
# --- SANDBOX TEST ---
# Let's just test with the first 3 teams to prove it works
TEST_SLUGS = TEAM_SLUGS[0:3] 
print(f"Testing with: {TEST_SLUGS}")

all_team_dfs = [] # This will hold each team's DataFrame

for slug in TEST_SLUGS:
    # This is our new, team-specific payroll URL
    TARGET_URL = f"https://www.spotrac.com/nba/{slug}/payroll/{SEASON_START_YEAR}/"
    
    print(f"Fetching {TARGET_URL} ...")
    
    try:
        response = requests.get(TARGET_URL, headers=headers)
        response.raise_for_status()
        
        # Use io.StringIO to avoid the warning
        tables = pd.read_html(io.StringIO(response.text))
        
        if tables:
            team_df = tables[0]
            # Add a column for the team to know where it came from (good for debugging)
            team_df['team_slug'] = slug 
            all_team_dfs.append(team_df)
        else:
            print(f"No tables found for {slug}")
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page for {slug}: {e}")
    
    time.sleep(1)

print("\n...Loop finished.")

Testing with: ['atlanta-hawks', 'boston-celtics', 'brooklyn-nets']
Fetching https://www.spotrac.com/nba/atlanta-hawks/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/boston-celtics/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/brooklyn-nets/payroll/2024/ ...

...Loop finished.


In [17]:
if all_team_dfs:
    # Combine all the separate team DataFrames into one big one
    combined_df_raw = pd.concat(all_team_dfs, ignore_index=True)
    
    print(f"Successfully combined {len(all_team_dfs)} tables.")
    print(f"Total rows found: {len(combined_df_raw)}")
    
    print("\n--- Head of combined table: ---")
    display(combined_df_raw.head())
    
    print("\n--- Raw Columns (will be messy): ---")
    print(combined_df_raw.columns)
else:
    print("No dataframes to combine.")
    combined_df_raw = None

Successfully combined 3 tables.
Total rows found: 53

--- Head of combined table: ---


,Player (15 of 15 + 3 of 3 TW),Pos,Age,Type,Cap Hit,Cap Hit Pct League Cap,Base Salary,Incentives Likely,Incentives Unlikely,Trade Bonus Proration,Guaranteed,team_slug,Player (15 of 15 + 2 of 3 TW)
0,Young Trae Young,PG,26,DRK-EXT,"$43,031,940",30.61%,"$43,031,940",-,-,-,"$43,031,940",atlanta-hawks,NaN
1,Capela Clint Capela,C,30,VET-EXT,"$22,265,280",15.84%,"$22,265,280",-,"$2,000,000",-,"$22,265,280",atlanta-hawks,NaN
2,LeVert Caris LeVert,SG,30,FA,"$16,615,384",11.82%,"$16,615,384",-,-,-,"$16,615,384",atlanta-hawks,NaN
3,Okongwu Onyeka Okongwu,C,24,RK-EXT,"$14,000,000",9.96%,"$14,000,000",-,-,-,"$14,000,000",atlanta-hawks,NaN
4,Risacher Zaccharie Risacher,SF,19,RK-1ST,"$12,569,040",8.94%,"$12,569,040",-,-,-,"$12,569,040",atlanta-hawks,NaN



--- Raw Columns (will be messy): ---
Index(['Player (15 of 15 + 3 of 3 TW)', 'Pos', 'Age', 'Type', 'Cap Hit',
       'Cap Hit Pct  League Cap', 'Base Salary', 'Incentives  Likely',
       'Incentives  Unlikely', 'Trade Bonus  Proration', 'Guaranteed',
       'team_slug', 'Player (15 of 15 + 2 of 3 TW)'],
      dtype='object')


In [ ]:
if combined_df_raw is not None:
    
    try:
        temp_df = combined_df_raw.copy()

        # "Flatten" the MultiIndex columns
        # e.g., ('Unnamed: 0_level_0', 'Player') becomes 'Unnamed: 0_level_0_Player'
        if isinstance(temp_df.columns, pd.MultiIndex):
            temp_df.columns = ['_'.join(col).strip() for col in temp_df.columns.values]

        print(f"\n--- Flattened Columns: --- \n{temp_df.columns.values}")

        player_col_name = 'Player (15 of 15 + 3 of 3 TW)' # Change this
        salary_col_name = 'Cap Hit'     # Change this
        
        print(f"\nUsing Player column: '{player_col_name}'")
        print(f"Using Salary column: '{salary_col_name}'")
        
        # Select only the columns we need
        final_df = temp_df[[player_col_name, salary_col_name]].copy()
        
        # Rename them to match our contract
        final_df.rename(columns={
            player_col_name: 'Player_Name',
            salary_col_name: 'Salary'
        }, inplace=True)
        
        # We need to remove duplicates. Some players (traded) might be on two teams.
        # We'll keep the first entry we find.
        final_df.drop_duplicates(subset=['Player_Name'], keep='first', inplace=True)

        print("\n--- Cleaned and Finalized DataFrame: ---")
        display(final_df.head())
        print(f"Total unique players: {len(final_df)}")

    except KeyError as e:
        print(f"\nError: A column name was wrong. You need to inspect the raw table.")
        print(f"Available columns are: {temp_df.columns}")
        final_df = None
else:
    print("Can't inspect, raw DataFrame is empty.")


--- Flattened Columns: --- 
['Player (15 of 15 + 3 of 3 TW)' 'Pos' 'Age' 'Type' 'Cap Hit'
 'Cap Hit Pct  League Cap' 'Base Salary' 'Incentives  Likely'
 'Incentives  Unlikely' 'Trade Bonus  Proration' 'Guaranteed' 'team_slug'
 'Player (15 of 15 + 2 of 3 TW)']

Using Player column: 'Player (15 of 15 + 3 of 3 TW)'
Using Salary column: 'Cap Hit'

--- Cleaned and Finalized DataFrame: ---


,Player_Name,Salary
0,Young Trae Young,"$43,031,940"
1,Capela Clint Capela,"$22,265,280"
2,LeVert Caris LeVert,"$16,615,384"
3,Okongwu Onyeka Okongwu,"$14,000,000"
4,Risacher Zaccharie Risacher,"$12,569,040"


Total unique players: 37


# ok, now full players for this year

In [20]:
import requests
import pandas as pd
from pathlib import Path
import io # To fix the FutureWarning
import time # To be a polite scraper

# This is the list of all 30 team slugs used by Spotrac
TEAM_SLUGS = [
    'atlanta-hawks', 'boston-celtics', 'brooklyn-nets', 'charlotte-hornets', 
    'chicago-bulls', 'cleveland-cavaliers', 'dallas-mavericks', 'denver-nuggets',
    'detroit-pistons', 'golden-state-warriors', 'houston-rockets', 'indiana-pacers',
    'los-angeles-clippers', 'los-angeles-lakers', 'memphis-grizzlies', 
    'miami-heat', 'milwaukee-bucks', 'minnesota-timberwolves', 'new-orleans-pelicans',
    'new-york-knicks', 'oklahoma-city-thunder', 'orlando-magic', 
    'philadelphia-76ers', 'phoenix-suns', 'portland-trail-blazers', 
    'sacramento-kings', 'san-antonio-spurs', 'toronto-raptors', 'utah-jazz', 
    'washington-wizards'
]

# Config vars for practice
SEASON = "2024-25"
SEASON_START_YEAR = SEASON.split('-')[0] # Will give us "2024"

PROJECT_ROOT = Path().resolve().parent.parent
RAW_SALARY_FILE = PROJECT_ROOT / "data" / "raw" / "raw_player_salaries.csv"

# Set browser headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

In [ ]:
print(f"Testing with: {TEAM_SLUGS}")

all_team_dfs = [] # This will hold each team's DataFrame

for slug in TEAM_SLUGS:
    # This is our new, team-specific payroll URL
    TARGET_URL = f"https://www.spotrac.com/nba/{slug}/payroll/{SEASON_START_YEAR}/"
    
    print(f"Fetching {TARGET_URL} ...")
    
    try:
        response = requests.get(TARGET_URL, headers=headers)
        response.raise_for_status()
        
        # Use io.StringIO to avoid the warning
        tables = pd.read_html(io.StringIO(response.text))
        
        if tables:
            team_df = tables[0]
            # Add a column for the team to know where it came from (good for debugging)
            team_df['team_slug'] = slug 
            all_team_dfs.append(team_df)
        else:
            print(f"No tables found for {slug}")
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page for {slug}: {e}")
    
    time.sleep(1)

print("\n..Loop finished.")

Testing with: ['atlanta-hawks', 'boston-celtics', 'brooklyn-nets', 'charlotte-hornets', 'chicago-bulls', 'cleveland-cavaliers', 'dallas-mavericks', 'denver-nuggets', 'detroit-pistons', 'golden-state-warriors', 'houston-rockets', 'indiana-pacers', 'los-angeles-clippers', 'los-angeles-lakers', 'memphis-grizzlies', 'miami-heat', 'milwaukee-bucks', 'minnesota-timberwolves', 'new-orleans-pelicans', 'new-york-knicks', 'oklahoma-city-thunder', 'orlando-magic', 'philadelphia-76ers', 'phoenix-suns', 'portland-trail-blazers', 'sacramento-kings', 'san-antonio-spurs', 'toronto-raptors', 'utah-jazz', 'washington-wizards']
Fetching https://www.spotrac.com/nba/atlanta-hawks/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/boston-celtics/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/brooklyn-nets/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/charlotte-hornets/payroll/2024/ ...
Fetching https://www.spotrac.com/nba/chicago-bulls/payroll/2024/ ...
Fetching https://www.spotrac.com/

In [ ]:
if combined_df_raw is not None:
    
    try:
        temp_df = combined_df_raw.copy()

        # "Flatten" the MultiIndex columns
        # e.g., ('Unnamed: 0_level_0', 'Player') becomes 'Unnamed: 0_level_0_Player'
        if isinstance(temp_df.columns, pd.MultiIndex):
            temp_df.columns = ['_'.join(col).strip() for col in temp_df.columns.values]

        print(f"\n--- Flattened Columns: --- \n{temp_df.columns.values}")

        player_col_name = 'Unnamed: 0_level_0_Player' # Change this
        salary_col_name = 'Base Salary_Base Salary'     # Change this
        
        print(f"\nUsing Player column: '{player_col_name}'")
        print(f"Using Salary column: '{salary_col_name}'")
        
        # Select only the columns we need
        final_df = temp_df[[player_col_name, salary_col_name]].copy()
        
        # Rename them to match our contract
        final_df.rename(columns={
            player_col_name: 'Player_Name',
            salary_col_name: 'Salary'
        }, inplace=True)
        
        # We need to remove duplicates. Some players (traded) might be on two teams.
        final_df.drop_duplicates(subset=['Player_Name'], keep='first', inplace=True)

        print("\n--- Cleaned and Finalized DataFrame: ---")
        display(final_df.head())
        print(f"Total unique players: {len(final_df)}")

    except KeyError as e:
        print(f"\nError: A column name was wrong. You need to inspect the raw table.")
        print(f"Available columns are: {temp_df.columns}")
        final_df = None
else:
    print("Can't inspect, raw DataFrame is empty.")


--- Flattened Columns: --- 
['Player (15 of 15 + 3 of 3 TW)' 'Pos' 'Age' 'Type' 'Cap Hit'
 'Cap Hit Pct  League Cap' 'Base Salary' 'Incentives  Likely'
 'Incentives  Unlikely' 'Trade Bonus  Proration' 'Guaranteed' 'team_slug'
 'Player (15 of 15 + 2 of 3 TW)']

Using Player column: 'Unnamed: 0_level_0_Player'
Using Salary column: 'Base Salary_Base Salary'

Error: A column name was wrong. You need to inspect the raw table.
Available columns are: Index(['Player (15 of 15 + 3 of 3 TW)', 'Pos', 'Age', 'Type', 'Cap Hit',
       'Cap Hit Pct  League Cap', 'Base Salary', 'Incentives  Likely',
       'Incentives  Unlikely', 'Trade Bonus  Proration', 'Guaranteed',
       'team_slug', 'Player (15 of 15 + 2 of 3 TW)'],
      dtype='object')
